# Ⅰ. Preface
## Ⅰ. 1. Backgrounds
 Nowadays, there are a lot of goods and those are saled in various channel. Especially sales in the online platform grows rapidly. So many companies have built their own website and mobile application for sale. <br>
 But in the point of view by customers, there are too many goods and services. So paradoxically, customers have to make an effort more when choosing what they want. If customers feel fatigue, they will leave. Besides, if the website or application is the new one, customers feel strange too. <br>
<br>
 Recommendation system receive attention as solution for the problem. From the past, Amazon has recommended books for each customes. And recently Youtube and Netflix recommend video which fit for each customers. As a result, customers need not meet the trouble with facing too lot of alternatives and finding what they want. It is possible by using big data and analyzing that.<br>
<br>
 With this background, we have built recommendation system for delivery appllication. Because of increasing single-person household and seeking convenience by people, delivery application expands rapidly in many countries. Especially state of COVID-19, with necessity for avoiding contact with other people, it becames more important. And If there is a recommendation function in delivery application, customers should order delivery service more easily. <br>

## Ⅰ. 2. Two main directions
### Ⅰ. 2. (1) Analysis by Quantitative Feedback(ratings)
 The first is based on rating by customers. Almost websites or applications have feedback system by customers after using goods or services. Customers give score according to their satisfaction. And company can use these information. <br>
 By **Collaborative Filtering(CF)**, the algorithm find other customer which show similar rating patern to certain customer. If goods or services which is bought by the similar customer has not bought yet, the algorithm recommend the goods for customer. It was basic logic in early era of recommedation system. In fact, nowadays it is not used mainly. <br>
 And **Matrix Fartorization(MF)** is method which find latent factor. By decomposing Matrix which contian information about customer and item to user latent matrix and item latent matrix, the algorithm predict rating matrix. That is, it also predicts rating for expected customer. But using all information is the different point to Collaborative Filtering, so its predition power is more strong. Especially, this model can be built with **Deep Learing(DL)**. With multi-layer analysis, accuracy of model can be increased. You can see it later.<br>
<br>
 The data "restaurant-recommendation-challenge" which is offered, contain information of ratings by customers to vendors. We have anlayzed this by above algorithm(CF, MF with DL), which is described later in part Ⅳ. <br>

### Ⅰ. 2. (2) Analysis by Contents
 The other approach is by features, called based on 'contents'. In other words, the algorithm finds items which have similar characteristics. <br>
 By calculating **Term Frequency - Inverse Document Frequency(TF-IDF)**, Similarity, like Cosine Similarity, can be obtained. TF means a value that how often certain words appear in a document. TF-IDF is multiply of TF and IDF. If this score is high, the word means that it appears often in this document not the other documents. **Cosine similarity** is a measure of similarity between two non-zero vectors of an inner product space. 
When a customer has bought a certain item, the company should recommend a similar item to the customer. And in this process, frequency of each word is calculated while excluding words which have low influence. <br>
 **Doc2Vec(Document Embedding with Paragraph Vectors)** is an extended algorithm in Word2Vec which predicts word by sequential paragraph analysis. The paragraph vector is the vector of the existing word vector (considering the words within the window size) plus the paragraph matrix.
By transposing documents to documents vector(embedding) and calculating Similarity between Documents, this algorithm recommends similar items. <br>
<br>
 In the data "restaurant-recommendation-challenge", We select two features as contents. One is 'tag(name)' which has ten fetures which describe each vendor, the other is an opening time which can describe the restaurant's target time slot. These 'contents' are analyzed by the above algorithm. That is, we consider these contents as one document, and apply a method to analyze the document. You can see it at part Ⅴ.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dot, Add, Flatten
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD, Adam, Adamax
from difflib import SequenceMatcher
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec

# Ⅱ. EDA

### Ⅱ.1. (1) train_full Datasets

First, look over the initial datasets, and simply visualize the data  

In [ ]:
#load data
data1=pd.read_csv("/kaggle/input/restaurant-recommendation-challenge/train_full.csv")
data1.head()

In [ ]:
data1.info()

The dataset has 73 columns and 5802400 rows :

So we just look over about 8-10 columns


*   gender : Customer's sex 
*   location type : Customer orders from one or more locations
*   language : Chose language
*   Opening Time : Vendr's operating time
*   city_id : City's id
*   vendor_rating : The vendor's average rating score 




In [ ]:
# Visualize missing values as a matrix
main_df=data1[['gender','location_type','language','OpeningTime','city_id','vendor_rating']]
msno.matrix(main_df)

In [ ]:
#Gender

print(data1['gender'].value_counts()) 
gender_null = np.count_nonzero(data1['gender'].isnull())
print(gender_null)

gender_null/data1.shape[0]
# variables : Male / Female 
# null ratio is about 30%

In [ ]:
sns.countplot('gender',data=data1)

In [ ]:
#location type
print(data1['location_type'].value_counts())
location_null = np.count_nonzero(data1['location_type'].isnull())
print(location_null)

print("null Ratio : ", location_null/data1.shape[0]) 
# variables : Home / Work / Other  but, we don't know the meaning of 'other' 
# null ratio is about 45%

In [ ]:
sns.countplot('location_type',data=data1)

In [ ]:
# The difference of location type by gender
sns.countplot(data1['gender'],hue=data1['location_type'])

In [ ]:
#Language
print(data1['language'].value_counts())
null = np.count_nonzero(data1['language'].isnull())
print(null)

null/data1.shape[0] 
# variables : it has only EN(english)
# null ratio is about 15%

In [ ]:
#Opening Time
null = np.count_nonzero(data1['OpeningTime'].isnull())
print(null)

null/data1.shape[0]
# null ratio is about 9%

In [ ]:
#Vendor's average rating score

print(data1['vendor_rating'].value_counts())
null = np.count_nonzero(data1['vendor_rating'].isnull())
print(null)

null/data1.shape[0]
# it has no NaN

In [ ]:
sns.countplot('vendor_rating',data=data1)

Vendor_rating's distribution is quite crowded between 4.2 and 4.5

### Ⅱ.1. (2) orders Datasets

And, I'll look over the other datasets about order information

In [ ]:
data2 = pd.read_csv("/kaggle/input/restaurant-recommendation-challenge/orders.csv")
data2.head()

In [ ]:
data2.info()

The dataset has 26 columns and 135303 rows :

Just simply explain some columns


*   akeed_order_id : Unique customer ID, used in train_locations and train_orders
*   vendor_rating : The ratings are rated by customers who use the vendor
*   vendor_id : vendor's unique id
*   customer_id : customer's unique id

In [ ]:
#vendor_rating

print(data2['vendor_rating'].value_counts())
null = np.count_nonzero(data2['vendor_rating'].isnull())
print(null)

null/data2.shape[0] 
# null ratio is about 66%

In [ ]:
sns.countplot('vendor_rating',data=data2)

The ratings except 0 score are almost 5 score

In [ ]:
#vendor_id
print(data2['vendor_id'].value_counts())
#the unique vendor count is 100

In [ ]:
#customer_id
print(data2['customer_id'].value_counts())
#the customer number is 27445

# Ⅲ. Preprocessing

## Ⅲ.1. Datasets Join
### Ⅲ.1.(1) For collaborative filtering

In [ ]:
# Choose some columns from data1(train_full.csv)
# vendor_rating - > mean_rating
dataset1 = data1[['customer_id','gender','location_type','id','OpeningTime','language','vendor_rating','serving_distance','vendor_tag_name','delivery_charge']]
dataset1.rename(columns = {"vendor_rating": "mean_rating"}, inplace = True)

# Make derived variables 'all' with id and customer_id columns
cols = ['customer_id', 'id']
dataset1['all'] = dataset1[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

dataset1.head()

In [ ]:
# Drop duplicates based 'all' derived variables
dataset1.drop_duplicates(['all'],inplace=True)

In [ ]:
# Choose some columns from data2(orders.csv)
# vendor_id - > id
dataset2 = data2[['akeed_order_id','customer_id','vendor_id', 'item_count', 'grand_total', 'vendor_rating']][:]
dataset2.rename(columns = {"vendor_id": "id"}, inplace = True)

# Make derived variables 'all' with id and customer_id columns
cols = ['customer_id', 'id']
dataset2['all'] = dataset2[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

dataset2.head()

In [ ]:
#The number of rows is dropped duplicate rows
print(dataset1.shape)
print(dataset2.shape)

Join dataset1 and dataset2

In [ ]:
df1=pd.merge(dataset1,dataset2,on='all',how='inner')
df1.head()

In [ ]:
df1.shape

In [ ]:
#Change some columns name and drop the same column
df1.rename(columns = {"customer_id_x": "customer_id"}, inplace = True)
df1.rename(columns = {"id_x": "vendor_id"}, inplace = True)
df1.drop(['customer_id_y','id_y'],axis=1,inplace=True)

We can check the number of rows is decreased by join the datsets

### Ⅲ.1.(2) For contents based

In [ ]:
# Choose some columns from dataset1(cleaned data)
df2=dataset1[['customer_id','id','vendor_tag_name']]
df2.rename(columns={'id':'vendor_id'},inplace=True)
df2.head()

## Ⅲ.2. Data Cleaning

In [ ]:
cols=[ 'serving_distance', 'delivery_charge','item_count', 'grand_total', 'vendor_rating']

def null_check(x):
 # print(df1_train[x].value_counts())
  null = np.count_nonzero(df1[x].isnull())
  print(null)

  return null/df1.shape[0] 

for i in cols:
  print(i,'null ratio :', null_check(i))

In [ ]:
#Drop language columns
df1.drop(['language'],axis=1,inplace=True)

#Remove null of gender columns
df1 = df1[df1['gender'].notnull()].reset_index(drop=True)

Modify 'gender' column

In [ ]:
#gender - > one-hot encoding (int)
sex=pd.get_dummies(df1["gender"], columns = ['gender'],prefix="sex",drop_first=True)

df1=pd.concat([df1,sex],axis=1)

#Drop the raw 'gender' column(char)
df1.drop(['gender'],axis=1,inplace=True)

In [ ]:
df1.rename(columns={'vendor_rating': 'rating'}, inplace=True)

In [ ]:
print(df1.shape)
df1.head()

In [ ]:
df1_train_for_anal = df1[:]

### Ⅲ.2.(1) For contents based

In [ ]:
train_for=df1[:]

In [ ]:
train_contents = train_for[['customer_id','vendor_id','OpeningTime','vendor_tag_name']]
train_contents.head()

Preprocessing the 'OpeningTime' column

In [ ]:
#Split 'Openingtime' column to 2 columns (open / close)
train_contents['OpeningTime'].fillna('-',inplace=True) 

time_split= train_contents.OpeningTime.str.split('-')
open=time_split.str.get(0)
close=time_split.str.get(1)

train_contents['Open']=open
train_contents['Close']=close

# Fill the blank to null value
train_contents['Open'].fillna('',inplace=True) 
train_contents['Close'].fillna('',inplace=True)

print(train_contents['Open'].unique())
print(train_contents['Close'].unique())

Make new columns 'morning, afternoon, evening' based open and close time

In [ ]:
def morning_func(x) :
  if x == "" :
    return None
  else :
    x1 = int(x[:2].replace(":", "").replace("a", ""))
    x2 = x[-2:]
    if (x1>=7 and x1 <= 10) and x2 == ("AM" or "am"): 
      return 1
    elif x1 <=10 and len(x) <= 2 :
      return 1
    else :
      return 0

def afternoon_func(x) :
  if x == "" :
    return None
  else :
    x1 = int(x[:2].replace(":", "").replace("a", ""))
    x2 = x[-2:]
    if x1 <= 1 and x2 == "PM": 
      return 1
    elif x1 == 12 and x2 == "PM":
      return 1
    elif x2 == ("AM" or "am"):
      return 1
    elif x1 <=10 and len(x) <= 2 :
      return 1
    else :
      return 0

def evening_func(x) :
  if x == "" :
    return None
  else :
    x1 = int(x[:2].replace(":",""))
    x2 = x[-2:]
    if (x1 >= 6 and x2 == "PM") or x2 == ("Am" or "am") :
      return 1
    elif x1 >= 22 and len(x)<=2:
      return 1      
    else :
      return 0

train_contents["morning"] = train_contents["Open"].apply(morning_func)
train_contents["afternoon"] = train_contents["Open"].apply(afternoon_func)
train_contents["evening"] = train_contents["Close"].apply(evening_func)

train_contents[:2]

If the 'breakfast' is in the vendor_tag_name and the morning value is null(blank), fill the morning and afternoon values to 1

In [ ]:
#Check the null ratio of 'vendor_tag_name' 
null = np.count_nonzero(train_contents['vendor_tag_name'].isnull())
print(null)
print(null/train_contents.shape[0]) #1%

#Remove the null value
train_contents= train_contents[train_contents['vendor_tag_name'].notnull()].reset_index(drop=True)
null = np.count_nonzero(train_contents['vendor_tag_name'].isnull())
print(null)

In [ ]:
# Cleaning 'vendor_tag_name'

#Change all of the chars to lower char  
train_contents['vendor_tag_name']=train_contents['vendor_tag_name'].apply(lambda x:x.lower())

# str -> list
train_contents['vendor_tag']= train_contents['vendor_tag_name'].str.split(',')
train_contents['vendor_tag'].head()

In [ ]:
#If 'breakfast' is in the tag, then add the morning
def breakfast1(tag,x2):
  if any('breakfast' in i for i in tag) and np.isnan(x2)==True :
    return 1
  # elif any('breakfast' in i for i in tag) and int(x2)==0 :  
  #   return 1
  else:
    return x2

train_contents['mor2']=train_contents.apply(lambda x:  breakfast1(x['vendor_tag'],x['morning']),axis=1) #아침 점심 저녁 다 추가 

#afternoon 1 / remainders are not changed
for i in range(len(train_contents['afternoon'])):
  if (np.isnan(train_contents['morning'][i])==True) and (train_contents['mor2'][i]==1.0) :
    train_contents['afternoon'][i]=1
  else:
    pass

#evening 0 / remainders are not changed
for i in range(len(train_contents['evening'])):
  if (np.isnan(train_contents['morning'][i])==True) and (train_contents['mor2'][i]==1.0) :
    train_contents['evening'][i]=0
  else:
    pass

In [ ]:
# Check and remove the null value
null = np.count_nonzero(train_contents['mor2'].isnull())
print(null)
print(null/train_contents.shape[0]) #0.4 % 

train_contents= train_contents[train_contents['mor2'].notnull()].reset_index(drop=True)
train_contents.drop(['morning'],axis=1,inplace=True)

train_contents.rename(columns={'mor2':'morning'},inplace=True)

In [ ]:
train_contents[:3]

# Ⅳ. Rating Based Model(Algorithm)

# Ⅳ. 1. Collaborative Filtering(CF)

### Ⅳ. 1. (1) Making Full Matrix

In [ ]:
# Extract Required variable for CF
cus_ven_ratings = df1_train_for_anal[['customer_id', 'vendor_id', 'rating']]
cus_ven_ratings

In [ ]:
# Calculate mean rating by only valid ratings(except missing and rated zero)
ratings_not_none = []

for i in range(0, cus_ven_ratings.shape[0]-1) :
  if pd.isnull(cus_ven_ratings.iloc[i][2]) == False and cus_ven_ratings.iloc[i][2] != 0 :
    ratings_not_none.append(cus_ven_ratings.iloc[i][2])
    
valid_rating_mean = np.mean(np.array(ratings_not_none))

In [ ]:
# Substitute missing and zero rating to mean by valid ratings
def rating_missing_func(x) :
  if pd.isnull(x) == True :
    return valid_rating_mean
  elif x == 0 :
    return valid_rating_mean
  else :
    return x

cus_ven_ratings["rating2"] = cus_ven_ratings["rating"].apply(rating_missing_func)
cus_ven_ratings

In [ ]:
# Reorganization dataframe(rename colums)
cus_ven_ratings = cus_ven_ratings[['customer_id', 'vendor_id', 'rating2']]
cus_ven_ratings.rename(columns={'rating2':'rating', 1:'customer_id_num'}, inplace=True)
cus_ven_ratings

In [ ]:
# Integration into individual ratings by group mean
cus_ven_ratings_mean = cus_ven_ratings.groupby(['customer_id', 'vendor_id']).mean()
cus_ven_ratings_mean

In [ ]:
df_cus_ven_ratings_mean = cus_ven_ratings_mean.reset_index()
df_cus_ven_ratings_mean

In [ ]:
# Making Full Matrix(Sparse Matrix)
rating_full_matrix = df_cus_ven_ratings_mean.pivot(index='customer_id', columns='vendor_id', values='rating')
rating_full_matrix

In [ ]:
# Calculate Similarity all pair of customers from Full Matirx
from sklearn.metrics.pairwise import cosine_similarity
rating_matrix_dummy = rating_full_matrix.copy().fillna(0)

customer_similarity = cosine_similarity(rating_matrix_dummy, rating_matrix_dummy)

customer_similarity = pd.DataFrame(customer_similarity, index = rating_full_matrix.index, columns=rating_full_matrix.index)
customer_similarity

### Ⅳ. 1. (2) Build CF Model

In [ ]:
# Function which calculate accuracy(Root Mean Squared Error)
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

In [ ]:
# Function which apply RMSE to CF model
def knn_score(model, neigbor_size=0) :
  id_pairs = zip(df_cus_ven_ratings_mean['customer_id'], df_cus_ven_ratings_mean['vendor_id'])
  y_pred = np.array([model(customer, vendor, neigbor_size) for (customer, vendor) in id_pairs])
  y_true = np.array(df_cus_ven_ratings_mean['rating'])
  return RMSE(y_true, y_pred)

In [ ]:
# CF model(restrict number of neighbor size)
def cf_knn(customer_id, vendor_id, neighbor_size=0):
    if vendor_id in rating_full_matrix:
        # Similarity of inputted customer and other customer
        sim_scores = customer_similarity[customer_id].copy()
        # Ratings by all customers for inputted vendor(restaurant)
        vendor_ratings = rating_full_matrix[vendor_id].copy()
        # Index of customers who are not rate inputted vendor
        none_rating_idx = vendor_ratings[vendor_ratings.isnull()].index
        # Exception rating(null) which of customers who are not rate inputted vendor
        vendor_ratings = vendor_ratings.drop(none_rating_idx)
        # Exception similarity which of customers who are not rate inputted vendor
        sim_scores = sim_scores.drop(none_rating_idx)
   
        # Case that neighbor size is not specified
        if neighbor_size == 0:          
            # Weighted mean of ratings by customers who rate inputted vendor
            mean_rating = np.dot(sim_scores, vendor_ratings) / sim_scores.sum()

        # Case that neighbor size is specified
        else:                       
            # Case that 2 or more people rate inputted vendor
            if len(sim_scores) > 1: 
                # Minimum value among inputted neighbor size and number of customers who rate inputted vendor
                neighbor_size = min(neighbor_size, len(sim_scores))
                # transpose to Numpy array for using argsort
                sim_scores = np.array(sim_scores)
                vendor_ratings = np.array(vendor_ratings)
                # Sorting similarity
                customer_idx = np.argsort(sim_scores)
                # Similarity as much as neighbor size
                sim_scores = sim_scores[customer_idx][-neighbor_size:]
                # Ratings as much as neighbor size
                vendor_ratings = vendor_ratings[customer_idx][-neighbor_size:]
                # Caculate final predicted ranting
                mean_rating = np.dot(sim_scores, vendor_ratings) / sim_scores.sum()
            else:
                # Substitute to valid mean in other case
                mean_rating = valid_rating_mean
    else:
        # Substitute to valid mean in other case
        mean_rating = valid_rating_mean
    return mean_rating 


In [ ]:
knn_score(cf_knn, neigbor_size=20)

### Ⅳ. 1. (3) Recommenadation for Customer by CF

In [ ]:
# Function which present recommendation list for certain customer by CF
def cf_recom_vendor(customer_id, n_items, neighbor_size=0):
    # Vendors which rated by inputted customer
    customer_vendor = rating_full_matrix.loc[customer_id].copy()
    
    for vendor in rating_full_matrix:
        # Excepton vendors which already rated by inputted customer
        if pd.notnull(customer_vendor.loc[vendor]):
            customer_vendor.loc[vendor] = 0
        # Calculate predicted rating about vendors which is not rated by inputted customer
        else:
            customer_vendor.loc[vendor] = cf_knn(customer_id, vendor, neighbor_size)
    
    # Sort vendors by predictted rating
    vendor_sort = customer_vendor.sort_values(ascending=False)[:n_items]
    recom_vendors_temp = df1_train_for_anal.loc[vendor_sort.index]
    recom_vendors_temp2 = recom_vendors_temp[['vendor_id', 'mean_rating', 'vendor_tag_name']]
    recom_vendors = recom_vendors_temp2.reset_index(drop=True)
    return recom_vendors

In [ ]:
# Example of recommendation list
cf_recom_vendor(customer_id='ZZV76GY', n_items=5, neighbor_size=30)

# Ⅳ. 2. Matrix Factorization with DeepLearning(MF with DL)

### Ⅳ. 2. (1) Making Full Matrix with index
For MF with DL, it is needed that Full Matrix composed with continuous numeric column name and row name.

In [ ]:
# Extract Required variable for MF
ratings = cus_ven_ratings

In [ ]:
# Integration into individual ratings by group mean
ratings = ratings.groupby(['customer_id', 'vendor_id']).mean().reset_index()

In [ ]:
# Make full matrix for temporary preprocessing
R_temp = ratings.pivot(index='customer_id', columns='vendor_id', values='rating').fillna(0)
R_temp

In [ ]:
# Mapping customer IDs to index(continuous numeric rowname)
customer_id_index = []

for i, one_id in enumerate(R_temp.T) :
  customer_id_index.append([one_id, i])

In [ ]:
df_customer_id_index = pd.DataFrame(customer_id_index)
df_customer_id_index.rename(columns={0:'customer_id', 1:'customer_idx'}, inplace=True)
df_customer_id_index

In [ ]:
# Mapping vendor IDs to index(continuous numeric columname)
vendor_id_index = []

for i, one_id in enumerate(R_temp) :
  vendor_id_index.append([one_id, i])

In [ ]:
df_vendor_id_index = pd.DataFrame(vendor_id_index)
df_vendor_id_index.rename(columns={0:'vendor_id', 1:'vendor_idx'}, inplace=True)
df_vendor_id_index

In [ ]:
# Merge rating and each index
ratings_with_index = pd.merge(ratings, df_customer_id_index, on='customer_id')
ratings_with_index = pd.merge(ratings_with_index, df_vendor_id_index, on='vendor_id')
ratings = ratings_with_index[['customer_idx', 'vendor_idx', 'rating']].astype(int)
ratings.rename(columns={'customer_id_num':'customer_idx', 'vendor_id_num':'vendor_idx', 'rating':'rating'}, inplace=True)
ratings

In [ ]:
# Making Full Matrix(Sparse Matrix)
rating_full_matrix_by_index_with_nan = ratings.pivot(index='customer_idx', columns='vendor_idx', values='rating')
rating_full_matrix_by_index_with_nan

In [ ]:
# Tranpose None to Zero
rating_full_matrix_by_index = ratings.pivot(index='customer_idx', columns='vendor_idx', values='rating').fillna(0)
rating_full_matrix_by_index

### Ⅳ. 2. (2) Build MF(with DL) Model

In [ ]:
# Number of latent factors
K = 100                    
# Total mean
mu = ratings.rating.mean()
# Number of customers
M = ratings.customer_idx.unique().shape[0]       
# Number of vendors
N = ratings.vendor_idx.unique().shape[0]      

# Function of calculating RMSE
def RMSE(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

# Embedding for Keras model
customer = Input(shape=(1, ))                                              
vendor = Input(shape=(1, ))                                               
P_embedding = Embedding(M, K, embeddings_regularizer=l2())(customer)      
Q_embedding = Embedding(N, K, embeddings_regularizer=l2())(vendor)        
customer_bias = Embedding(M, 1, embeddings_regularizer=l2())(customer)    
vendor_bias = Embedding(N, 1, embeddings_regularizer=l2())(vendor)        

# Layers
from tensorflow.keras.layers import Dense, Concatenate, Activation
P_embedding = Flatten()(P_embedding)                                   
Q_embedding = Flatten()(Q_embedding)                                   
customer_bias = Flatten()(customer_bias)                               
vendor_bias = Flatten()(vendor_bias)                                   
R = Concatenate()([P_embedding, Q_embedding, customer_bias, vendor_bias])

# Neural network for Deep Learning
R = Dense(2048)(R)
R = Activation('linear')(R)
R = Dense(256)(R)
R = Activation('linear')(R)
R = Dense(1)(R)

# Model compile
model = Model(inputs=[customer, vendor], outputs=R)
model.compile(
  loss=RMSE,
  optimizer=Adamax(),
  metrics=[RMSE]
)
model.summary()

In [ ]:
# Model fitting
result = model.fit(
  x=[ratings.customer_idx.values, ratings.vendor_idx.values],
  y=ratings.rating.values - mu,
  epochs=10,
  batch_size=512,
  validation_data=(
    [ratings.customer_idx.values, ratings.vendor_idx.values],
    ratings.rating.values - mu
  )
)

In [ ]:
# Plot of RMSE
import matplotlib.pyplot as plt
plt.plot(result.history['RMSE'], label="RMSE")
plt.xlabel('epoch')
plt.ylabel('RMSE')
plt.legend()
plt.show()

In [ ]:
# Comparision of Actual - Predicted rating
customer_ids = ratings.customer_idx.values[0:6]
vendor_ids = ratings.vendor_idx.values[0:6]
predictions = model.predict([customer_ids, vendor_ids]) + mu
print("Actuals: \n", ratings[0:6])
print()
print("Predictions: \n", predictions)

### Ⅳ. 2. (3) Recommenadation for Customer by MF

In [ ]:
def recom_vendor(customer_idx, n_items):
    # Vendors which rated by inputted customer
    customer_vendor = rating_full_matrix_by_index_with_nan.loc[customer_idx].copy()
    
    for vendor in rating_full_matrix_by_index_with_nan:
        # Excepton vendors which already rated by inputted customer
        if pd.notnull(customer_vendor.loc[vendor]):
            customer_vendor.loc[vendor] = 0
        # Calculate predicted rating about vendors which is not rated by inputted customer
        else:
            customer_vendor.loc[vendor] = round(min(model.predict([np.array([customer_idx]), np.array([vendor])])[0][0] + mu, 5), 3)
    
    # Sort vendors by predictted rating
    vendor_sort = customer_vendor.sort_values(ascending=False)[:n_items]   
    df_vendor_sort = pd.DataFrame(vendor_sort)    
    df_vendor_sort.rename(columns={'vendor_idx':'vendor_idx', customer_idx:'predicted_rating'}, inplace=True)

    return df_vendor_sort

In [ ]:
# Remapping customer IDs and customer index
customer_id_idx = ratings_with_index[['customer_id', 'customer_idx']]
customer_id_idx = customer_id_idx.drop_duplicates()

In [ ]:
# Remapping vendor IDs and vendor index
vendor_id_idx = ratings_with_index[['vendor_id', 'vendor_idx']]
vendor_id_idx = vendor_id_idx.drop_duplicates()

In [ ]:
# Preparing dataframe for extracting vendor tag
mf_df1 = df1_train_for_anal[['vendor_id', 'mean_rating', 'vendor_tag_name']]
mf_df1 = mf_df1.drop_duplicates()

In [ ]:
# Function which present recommendation list for certain customer by MF with DL
def mf_dl_recom_vendor_list(customer_id, n_items) :
  df_specified_customer = customer_id_idx[customer_id_idx['customer_id']== customer_id]
  specified_customer_idx = df_specified_customer.iloc[0][1]
  mf_recom_list_temp = recom_vendor(customer_idx = specified_customer_idx, n_items = n_items)
  mf_recom_list_temp2 = pd.merge(mf_recom_list_temp, vendor_id_idx, how='inner', on='vendor_idx')
  mf_recom_list_temp3 = mf_recom_list_temp2[['vendor_id', 'predicted_rating']]
  
  mf_recom_list = pd.merge(mf_recom_list_temp3, mf_df1, how='inner', on='vendor_id')

  return mf_recom_list

In [ ]:
# Example of recommendation list
mf_dl_recom_vendor_list(customer_id = 'ZZV76GY', n_items = 5)

# Ⅴ. Contents Based Recommendation System


In [ ]:
df1_contents_for_anal=train_contents[:]

In [ ]:
df1_contents_for_anal.head()

In [ ]:
df1_contents_for_anal['vendor_tag']= df1_contents_for_anal['vendor_tag_name'].str.split(',')

#strip
df1_contents_for_anal['vendor_tag']=df1_contents_for_anal['vendor_tag'].apply(lambda x:[str.lower(i.replace(" ",""))for i in x])

In [ ]:
#Check similar word
def similar(a, b):
    ratio=SequenceMatcher(None, a, b).ratio()
    return print("Similarity of {} and {}  : {}".format(a,b,ratio) )

similar('pasta','pastas')
similar('pasta','pastry')
similar('pizza','pizzas')
similar('soups','shuwa')
similar('shawarma','shuwa')
similar('thali','thai')
similar('milkshakes','mishkak')

#change the word when similarity is over than 0.8 
df1_contents_for_anal['vendor_tag']=df1_contents_for_anal['vendor_tag'].apply(lambda x:[ i.replace("pastas","pasta")for i in x])
df1_contents_for_anal['vendor_tag']=df1_contents_for_anal['vendor_tag'].apply(lambda x:[ i.replace("pizzas","pizza")for i in x])
df1_contents_for_anal['vendor_tag']=df1_contents_for_anal['vendor_tag'].apply(lambda x:[ i.replace("thali","thai")for i in x])

df1_contents_for_anal['vendor_tag1']=df1_contents_for_anal['vendor_tag'].apply(lambda x:' '.join(x))

In [ ]:
df1_contents_for_anal.head()

In [ ]:
df1_contents_for_anal['vendor_id'].value_counts()

In [ ]:
prac= df1_contents_for_anal.drop_duplicates("vendor_id", keep="first", inplace=False)
print(prac.shape)

prac['vendor_id']=prac['vendor_id'].astype(str)
prac1=prac[:]

# V. 1. TF - IDF model
### V.1.(1). TF - IDF base model

In [ ]:
prac.set_index('vendor_id',inplace=True)
prac.head(2)

In [ ]:
vectorizer = TfidfVectorizer()
count_matrix = vectorizer.fit_transform(prac['vendor_tag1'])
print(vectorizer.get_feature_names())
print(vectorizer.vocabulary_)

In [ ]:
indices = pd.Series(prac.index)
indices[:5]

In [ ]:
count_matrix

In [ ]:
cosine_sim = cosine_similarity(count_matrix,count_matrix)
print(cosine_sim)

In [ ]:
indices[indices == '113'].index[0]

In [ ]:
list(enumerate(cosine_sim[0]))

In [ ]:
prac=prac.reset_index()

def get_recommendations(id, cosine_sim=cosine_sim):
    indices = pd.Series(prac.index, index = prac['vendor_id']).drop_duplicates() 
    # get index from vendor_id
    idx = indices[id]

    # cosin_similarity
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # most similar vendor_id (10)
    sim_scores = sim_scores[1:11]

    return sim_scores

In [ ]:
get_recommendations('113')

The vendor id '113' is most similar with '36' (cosine similarity is about 0.71)

### V.1.(2) Adding time tag (morning/afternoon/evening)

In [ ]:
cols=['afternoon','evening','morning']

prac['time']='' # to create an empty column
for col_name in cols:
    prac.loc[prac[col_name]==1,'time']= prac['time']+' '+col_name

prac=prac[['vendor_id'	,'customer_id',	'vendor_tag','vendor_tag1','time']]
prac.head()

In [ ]:
#'time' column : str -> list
prac['time']=prac['time'].str.split(' ')

def remove_blank(lists):
  return [key for key in lists if key !='']

prac['time']=prac['time'].apply(lambda x:remove_blank(x))
prac['time'][0]

In [ ]:
# combine 'vendor_tag' with 'time' (str for w2v)
prac['time1']=prac['time'].apply(lambda x:' '.join(x))
prac['time_tag']=prac[['vendor_tag1','time1']].apply(lambda x: ' '.join(x),axis=1)

prac=prac[['vendor_id','customer_id','time_tag','vendor_tag','vendor_tag1']]
prac.head(2)
prac2=prac[:]

In [ ]:
vectorizer = TfidfVectorizer()
count_matrix = vectorizer.fit_transform(prac['time_tag'])
print(vectorizer.get_feature_names())
print(vectorizer.vocabulary_)

In [ ]:
prac.set_index('vendor_id',inplace=True)

indices = pd.Series(prac.index)
cosine_sim = cosine_similarity(count_matrix,count_matrix)
cosine_sim.shape

In [ ]:
prac=prac.reset_index()

def item_recommendations(id, cosine_sim=cosine_sim):
    indices = pd.Series(prac.index, index = prac['vendor_id']).drop_duplicates() 
    idx = indices[id]

    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]

    return sim_scores

In [ ]:
item_recommendations('113')

The vendor id '113' is most similar with '36' (cosine similarity is about 0.76)
So, we can see the cosine similarity is imporved a little bit

# V. 2. Doc2Vec model
### V. 2. (1). Word2Vec for Doc2Vec

In [ ]:
prac1.head(2)

In [ ]:
prac1.set_index('vendor_id',inplace=True)
#Embedding the word using word2vec
corpus=prac1['vendor_tag']
model = Word2Vec(size=4, window=1, min_count=1, workers=4)
model.build_vocab(corpus)

word_vectors = model.wv
vocabs = word_vectors.vocab.keys()
vocabs

In [ ]:
word_vectors.most_similar('breakfast')

The words are that similar word with 'breakfast'

### V. 2. (2). Doc2Vec

In [ ]:
# Mean of word vector
def vectors(document_list):
    document_embedding_list = []

    for line in document_list:
        doc2vec = None
        count = 0
        for word in line.split():
            if word in model.wv.vocab:
                count += 1
                if doc2vec is None:
                    doc2vec = model[word]
                else:
                    doc2vec = doc2vec + model[word]

        if doc2vec is not None:
            doc2vec = doc2vec / count
            document_embedding_list.append(doc2vec)
    return document_embedding_list

document_embedding_list = vectors(prac1['vendor_tag1'])
print('Number of document vector:',len(document_embedding_list))

In [ ]:
cosine_sim = cosine_similarity(document_embedding_list, document_embedding_list)

print('the shape of cosine similarity matrix :',cosine_sim.shape)

In [ ]:
prac1=prac1.reset_index()
def get_recommendations_w2v(id, cosine_sim=cosine_sim):
    indices = pd.Series(prac1.index, index = prac1['vendor_id']).drop_duplicates() 

    idx = indices[id]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:11]

    return sim_scores

In [ ]:
get_recommendations_w2v('113')

### V. 2. (3). Doc2Vec with time tag

In [ ]:
prac1.head()

In [ ]:
prac2['time_tag_list']= prac2['time_tag'].str.split(' ')

#word2vec을 이용해 단어 임베딩
corpus=prac2['time_tag_list']

model = Word2Vec(size=4, window=1, min_count=1, workers=4)
model.build_vocab(corpus)

word_vectors = model.wv
vocabs = word_vectors.vocab.keys()
vocabs

In [ ]:
word_vectors.most_similar('breakfast')

In [ ]:
def vectors(document_list):
    document_embedding_list = []

    for line in document_list:
        doc2vec = None
        count = 0
        for word in line.split():
            if word in model.wv.vocab:
                count += 1
                if doc2vec is None:
                    doc2vec = model[word]
                else:
                    doc2vec = doc2vec + model[word]

        if doc2vec is not None:
            doc2vec = doc2vec / count
            document_embedding_list.append(doc2vec)
    return document_embedding_list

document_embedding_list = vectors(prac2['time_tag'])
print('the number of document vector:',len(document_embedding_list))

cosine_sim = cosine_similarity(document_embedding_list, document_embedding_list)
print('the shape of cosine similarity matrix :',cosine_sim.shape)

indices = pd.Series(prac.index, index=prac2['vendor_id']).drop_duplicates()

def get_recommendations_w2v_time(id, cosine_sim=cosine_sim):
    indices = pd.Series(prac.index, index = prac2['vendor_id']).drop_duplicates() 

    idx = indices[id]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]

    return sim_scores

In [ ]:
get_recommendations_w2v_time('113')

As you know, model performance is best at the Doc2Vec with time tag according to the cosine similarity. <br>
Doc2Vec with time tag > Doc2Vec > Tf-Idf with time tag > Tf-Idf<br>
In this order, the model performance is good in contents based recommendation system.

# Ⅵ. Conclusion
## Ⅵ. 1. Expected effects
### Ⅵ. 1. (1) to Delivery Application
 Nowadays, there are many delivery applications. And some of them show restaurant lists which fit each user. For their main business model is to get fee from customers and vendors, more transactions means more revenue. So it is important to attract customers using recommendation lists.<br>
<br>
### Ⅵ. 1. (2) to Customers
 When customers access the list, it is easy to choose to eat and to order that. It means decrease of bounce rate. Users will not just see and exit the application anymore.<br>
<br>
### Ⅵ. 1. (3) to Vendors
 And to restaurants, the system makes them get better feedback not only quantitative but also qualitative. Moreover, it can be an accelerator to secure regular customers. <br>


## Ⅵ. 2. Expendability
### Ⅵ. 2. (1) Combination of two directions
 Rating based models recommend restaurants by using information of each customer, rather than contents based models using information of each item(vendors). So, the company can show lists by rating based model when user runs the application. And if customers select certain items(or add to cart), lists of similar items by contents based model can be effective.


### Ⅵ. 2. (2) Limitations of analysis
 For more accurate analysis, the model can have more various variables as a feature. For example, the situation of order should affect the selection of restaurants. Order at home and office should be different. And location of customers and vendors is also an important factor. If the function is included, the problem of cold start cases is solved. However, our models exclude that because of imperfection of data.
